In [196]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from pathlib import Path
from io import StringIO

sns.set_style('whitegrid')
plt.rcParams["figure.figsize"] = (15, 5)

In [197]:
LOG_DIRECTORY = Path('log/')
assert LOG_DIRECTORY.exists()
log_files = sorted(LOG_DIRECTORY.iterdir())
log_files

[PosixPath('log/bot_logs.txt'),
 PosixPath('log/launcher_logs.txt'),
 PosixPath('log/market.txt'),
 PosixPath('log/orderbook.txt'),
 PosixPath('log/orders.txt'),
 PosixPath('log/our_trades.txt'),
 PosixPath('log/positions.txt'),
 PosixPath('log/runner.txt'),
 PosixPath('log/strategy.txt'),
 PosixPath('log/strategy_logs.txt'),
 PosixPath('log/target_bid_ask.txt'),
 PosixPath('log/trades.txt')]

In [198]:
def format_number(n: float):
    n = round(n)
    return f"{n:,}".replace(',', ' ')

In [199]:
PX_STEP = 0.5


class StrategyState:
    def __init__(self) -> None:
        self.update()

    def update(self):
        self.trades = self.load_log_pandas("trades")
        self.orderbook = self.load_log_pandas("orderbook")
        self.orders = self.process_orders(self.load_log_pandas("orders"))
        self.our_trades = self.load_log_pandas("our_trades")
        self.positions = self.load_log_pandas("positions")
        self.target_bid_ask = self.process_target_bid_ask(self.load_log_pandas("target_bid_ask"))

    @staticmethod
    def load_log_pandas(file: str) -> pd.DataFrame:
        log_file = LOG_DIRECTORY / f"{file}.txt"
        assert log_file.exists(), log_file
        with open(log_file) as f:
            lines = f.readlines()
        header = lines[0]
        try:
            ind = lines.index(header, 1)
            lines = lines[:ind]
        except ValueError:
            pass
        df = pd.read_csv(StringIO("".join(lines)))
        for col in ["strategy_time", "exchange_time"]:
            if col in df.columns:
                df[col] = pd.to_datetime(df[col])

        for col in df.columns:
            if col.startswith("bid_px_") or col.startswith("ask_px_") or col == "px" or col == "money":
                df[col] = df[col] * PX_STEP
        return df

    @staticmethod
    def process_orders(orders: pd.DataFrame) -> pd.DataFrame:
        orders['key'] = -orders['px'] * ((orders['direction'] == 'Buy') * 2 - 1)
        orders = orders.sort_values(by=['internal_log_id', 'key'])
        return orders.drop(columns=['key'])

    @staticmethod
    def process_target_bid_ask(target_bid_ask: pd.DataFrame) -> pd.DataFrame:
        bid_quotes = target_bid_ask["bid_count;bid_px;bid_qty"].str.split(";")
        target_bid_ask["bid_count"] = bid_quotes.str[0]
        target_bid_ask["bid_px"] = bid_quotes.str[1::2].apply(lambda x: [int(i) * PX_STEP for i in x])
        target_bid_ask["bid_qty"] = bid_quotes.str[2::2].apply(lambda x: [int(i) for i in x])
        assert (target_bid_ask["bid_px"].str.len() == target_bid_ask["bid_qty"].str.len()).all()

        ask_quotes = target_bid_ask["ask_count;ask_px;ask_qty"].str.split(";")
        target_bid_ask["ask_count"] = ask_quotes.str[0]
        target_bid_ask["ask_px"] = ask_quotes.str[1::2].apply(lambda x: [int(i) * PX_STEP for i in x])
        target_bid_ask["ask_qty"] = ask_quotes.str[2::2].apply(lambda x: [int(i) for i in x])
        assert (target_bid_ask["ask_px"].str.len() == target_bid_ask["ask_qty"].str.len()).all()

        target_bid_ask = target_bid_ask.drop(columns=["bid_count;bid_px;bid_qty", "ask_count;ask_px;ask_qty"])
        target_bid_ask["function"] = target_bid_ask["msg"].str.split("(").str[0]
        target_bid_ask["params"] = target_bid_ask["msg"].str.extract(r"\((.*?)\)")
        target_bid_ask = target_bid_ask.drop(columns=["msg"])

        columns = ["strategy_time", "when", "function", "params", "bid_count", "ask_count", "bid_px", "bid_qty", "ask_px", "ask_qty"]
        assert set(columns) == set(target_bid_ask.columns), f"{set(target_bid_ask.columns) - set(columns)}"
        return target_bid_ask[columns]


state = StrategyState()

In [ ]:
def plot_latency(df, title: str, left: pd.Timestamp, right: pd.Timestamp):
    v = (df["strategy_time"] - df["exchange_time"]).dt.total_seconds()
    v.index = df["exchange_time"]
    plt.scatter(v.index, v)
    plt.title(f"{title} Latency")
    plt.ylabel("(strategy_time - exchange_time) seconds")
    plt.xlabel("Time")
    plt.xlim(left, right)
    plt.show()


def plot_latencies(state: StrategyState):
    orderbook = state.orderbook.iloc[40:]
    trades = state.trades.iloc[5:]
    left = min(orderbook["exchange_time"].iloc[0], trades["exchange_time"].iloc[0])
    right = max(orderbook["strategy_time"].iloc[-1], trades["strategy_time"].iloc[-1])
    diff = right - left
    right += diff * 0.01
    left -= diff * 0.01
    plot_latency(orderbook, "OrderBook", left=left, right=right)
    plot_latency(trades, "Trades", left=left, right=right)


plot_latencies(state)

In [ ]:
def plot_orderbook_with_trades(state: StrategyState, our_trades_size=300):
    # OrderBook
    orderbook = state.orderbook
    plt.step(orderbook["exchange_time"], orderbook["bid_px_0"], color="green", label="best_bid", linestyle="--", where='post')
    plt.step(orderbook["exchange_time"], orderbook["ask_px_0"], color="red", label="best_ask", linestyle="--", where='post')

    # Trades
    trades = state.trades
    buy_trades = trades[trades["direction"] == "Buy"]
    sell_trades = trades[trades["direction"] == "Sell"]
    plt.scatter(buy_trades["exchange_time"], buy_trades["px"], color="green", alpha=0.5, label="Buy Market Trades")
    plt.scatter(sell_trades["exchange_time"], sell_trades["px"], color="red", alpha=0.5, label="Sell Market Trades")

    # Our Trades
    our_trades = state.our_trades
    buy_our_trades = our_trades[our_trades["direction"] == "Buy"]
    sell_our_trades = our_trades[our_trades["direction"] == "Sell"]
    # TODO: parse exchange_time

    plt.scatter(buy_our_trades["strategy_time"], buy_our_trades["px"], color="green", marker="^", s=our_trades_size, alpha=0.5, label="Buy Our Trades")
    plt.scatter(sell_our_trades["strategy_time"], sell_our_trades["px"], color="red", marker="v", s=our_trades_size, alpha=0.5, label="Sell Our Trades")

    plt.ylabel("Price")
    plt.xlabel("Time")
    plt.title("OrderBook with trades")
    plt.legend()


plot_orderbook_with_trades(state)

In [ ]:
def plot_our_orders(state: StrategyState, drop_first: int = 250, max_orders: int = 5, alpha_our_orders: float = 0.05):
    orders = state.orders.iloc[drop_first:]
    orders = orders.groupby(["internal_log_id", "direction"])[orders.columns].apply(lambda x: x.head(max_orders))
    buy_orders = orders[orders["direction"] == "Buy"]
    sell_orders = orders[orders["direction"] == "Sell"]
    plt.scatter(buy_orders["strategy_time"], buy_orders["px"], color="green", alpha=alpha_our_orders, marker="s", label="Buy orders")
    plt.scatter(sell_orders["strategy_time"], sell_orders["px"], color="red", alpha=alpha_our_orders, marker="s", label="Sell orders")
    plt.legend()


plot_orderbook_with_trades(state)
plot_our_orders(state)

In [ ]:
def merge_timeseries(*dfs: pd.DataFrame, fillna: bool = True) -> pd.DataFrame:
    """
    Merge several time series by index
    For each index in dfs take the latest observation from each df
    The result may have NaNs at the beginning of some columns
    """
    df = dfs[0]
    for i in range(1, len(dfs)):
        df = pd.merge(df, dfs[i], how="outer", left_index=True, right_index=True)
    if fillna:
        df = df.ffill()  # Forward fill with latest available value
    return df


def plot_positions(state: StrategyState):
    positions = state.positions.set_index("strategy_time")
    orderbook = state.orderbook
    mid_px = pd.DataFrame((orderbook["bid_px_0"] + orderbook["ask_px_0"]) / 2, columns=["mid_px"])
    mid_px.index = orderbook["exchange_time"]

    df = merge_timeseries(mid_px, positions).dropna()
    capital = df["money"] + df["qty"] * df["mid_px"]

    plt.subplots(2, 1, figsize=(15, 8))
    plt.subplot(2, 1, 1)

    upper_qty = math.ceil(capital.iloc[0] / df["mid_px"].iloc[0])
    lower_qty = 0
    plt.step(df.index, (df["qty"] - lower_qty) / (upper_qty - lower_qty) * 100, linestyle="--", where="post")
    plt.axhline(100, color="black", linestyle="--", alpha=0.5, label=f"Quantity constraints: [{lower_qty}, {upper_qty}]")
    plt.axhline(0, color="black", linestyle="--", alpha=0.5)

    upper_money = capital.iloc[0]
    lower_money = 0.0
    plt.step(df.index, (df["money"] - lower_money) / (upper_money - lower_money) * 100, linestyle="--", where="post")
    plt.axhline(100, color="black", linestyle="--", alpha=0.5, label=f"Money constraints: [{lower_money} RUB, {format_number(upper_money)} RUB]")
    plt.axhline(0, color="black", linestyle="--", alpha=0.5)
    plt.legend(loc="upper left")

    plt.title("Quantity and Money, %")
    plt.ylabel("%")
    plt.xlabel("Time")

    plt.subplot(2, 1, 2)
    plt.step(df.index, capital / capital.iloc[0] * 100 - 100, linestyle="--", where="post", label="Capital")
    plt.step(mid_px.index, mid_px / mid_px.iloc[0] * 100 - 100, linestyle="--", where="post", label="Price")
    plt.title("Capital and Price Change (%)")
    plt.ylabel("%")
    plt.xlabel("Time")
    plt.legend()

    plt.tight_layout()


plot_positions(state)

In [204]:
state.orders.to_csv("tmp/orders.csv")
state.target_bid_ask.to_csv("tmp/target_bid_ask.csv")

In [217]:
# state.orders.tail(10)

In [216]:
# for internal_log_id, orders in state.orders.groupby("internal_log_id"):
#     if internal_log_id < 151:
#         continue
#     next_target_bid_ask = state.target_bid_ask.copy()
#     ind_right = np.searchsorted(next_target_bid_ask["strategy_time"], orders["strategy_time"].iloc[-1], side="right")
#     next_target_bid_ask = next_target_bid_ask.iloc[ind_right: ind_right + 2]
#     next_bid_count = next_target_bid_ask.bid_count.iloc[-1]
#     next_ask_count = next_target_bid_ask.ask_count.iloc[-1]
#     orders = orders.groupby(['px', 'direction']).agg({'internal_log_id': 'min', 'strategy_time': 'min', 'qty': 'sum'})
#     n_quotes = orders.index.value_counts().groupby("direction").count().to_dict()
#     n_bids = n_quotes.get("Buy", 0)
#     n_asks = n_quotes.get("Sell", 0)
#     print(n_bids, n_asks)
#     print(next_bid_count, next_ask_count)
#     display(orders)
#     display(next_target_bid_ask)
#     break

In [215]:
tmp = state.target_bid_ask.iloc[5]
max_qty = sum(tmp['bid_qty'] + tmp['ask_qty'])
max_qty

75

In [226]:
positions = state.positions
before = state.target_bid_ask.iloc[150::2]
after = state.target_bid_ask.iloc[151::2]
total_money_list = []
for (_, b), (_, a) in zip(before.iterrows(), after.iterrows()):
    ind = np.searchsorted(positions["strategy_time"], b["strategy_time"], side="right")
    p = positions.iloc[ind - 1]

    qty = p['qty']
    money = p['money']
    sell_money = sum([p * q for p, q in zip(a['ask_px'], a['ask_qty'])])
    total_money = money + sell_money
    total_money_list.append(total_money)

    assert money >= 0
    assert sum(a['ask_qty']) == qty
    assert sum(a['ask_qty']) + sum(a['bid_qty']) == max_qty
    # break

In [229]:
total_money_list[0] - total_money_list[-1]

555.0

In [253]:
# order_id = 45755265579; exists on internal_log_id = 139; executes at id = 140
# px = 6160 (3080.0)
# Place again at id = 141
state.orders[state.orders.internal_log_id == 139]

,internal_log_id,strategy_time,order_id,direction,px,qty
3341,139,2024-04-12 19:39:05.704271465,45755265925,Buy,3081.0,2
3342,139,2024-04-12 19:39:05.704282165,45755266008,Buy,3081.0,1
3338,139,2024-04-12 19:39:05.704237865,45755265609,Buy,3080.5,1
3339,139,2024-04-12 19:39:05.704249565,45755265706,Buy,3080.5,1
3340,139,2024-04-12 19:39:05.704260665,45755265825,Buy,3080.5,1
3337,139,2024-04-12 19:39:05.704223865,45755265579,Buy,3080.0,3
3335,139,2024-04-12 19:39:05.704185165,45755236991,Sell,3082.5,2
3336,139,2024-04-12 19:39:05.704196365,45755237080,Sell,3082.5,1
3334,139,2024-04-12 19:39:05.704174365,45755221507,Sell,3083.0,3
3333,139,2024-04-12 19:39:05.704163565,45755217406,Sell,3083.5,3


In [251]:
state.target_bid_ask[state.target_bid_ask['strategy_time'] >= pd.to_datetime('2024-04-12 19:40:26.370759803')]

,strategy_time,when,function,params,bid_count,ask_count,bid_px,bid_qty,ask_px,ask_qty
6122,2024-04-12 19:40:26.371409403,before,UpdateTargetQuotesOnExecution,bid; px=6160; qty=3,3,22,"[3081.0, 3080.5, 3080.0]","[3, 3, 3]","[3082.5, 3083.0, 3083.5, 3084.0, 3084.5, 3085....","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ..."
6123,2024-04-12 19:40:26.371501203,after,UpdateTargetQuotesOnExecution,bid; px=6160; qty=3,2,23,"[3080.5, 3080.0]","[3, 3]","[3082.0, 3082.5, 3083.0, 3083.5, 3084.0, 3084....","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ..."
6124,2024-04-12 19:40:27.169391816,before,UpdateTargetQuotesOnExecution,bid; px=6161; qty=1,2,23,"[3080.5, 3080.0]","[3, 3]","[3082.0, 3082.5, 3083.0, 3083.5, 3084.0, 3084....","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ..."
6125,2024-04-12 19:40:27.169438016,after,UpdateTargetQuotesOnExecution,bid; px=6161; qty=1,2,24,"[3080.5, 3080.0]","[2, 3]","[3081.5, 3082.0, 3082.5, 3083.0, 3083.5, 3084....","[1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ..."
6126,2024-04-12 19:40:27.978385333,before,UpdateTargetQuotesOnExecution,bid; px=6161; qty=1,2,24,"[3080.5, 3080.0]","[2, 3]","[3081.5, 3082.0, 3082.5, 3083.0, 3083.5, 3084....","[1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ..."
6127,2024-04-12 19:40:27.978429833,after,UpdateTargetQuotesOnExecution,bid; px=6161; qty=1,2,24,"[3080.5, 3080.0]","[1, 3]","[3081.5, 3082.0, 3082.5, 3083.0, 3083.5, 3084....","[2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ..."
6128,2024-04-12 19:40:28.459293343,before,UpdateTargetQuotesOnExecution,bid; px=6162; qty=2,2,24,"[3080.5, 3080.0]","[1, 3]","[3081.5, 3082.0, 3082.5, 3083.0, 3083.5, 3084....","[2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ..."
6129,2024-04-12 19:40:28.459341843,after,UpdateTargetQuotesOnExecution,bid; px=6162; qty=2,1,25,[3080.0],[2],"[3081.0, 3081.5, 3082.0, 3082.5, 3083.0, 3083....","[1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ..."
6130,2024-04-12 19:40:28.822508050,before,UpdateTargetQuotesOnExecution,bid; px=6162; qty=1,1,25,[3080.0],[2],"[3081.0, 3081.5, 3082.0, 3082.5, 3083.0, 3083....","[1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ..."
6131,2024-04-12 19:40:28.822630150,after,UpdateTargetQuotesOnExecution,bid; px=6162; qty=1,1,25,[3080.0],[1],"[3081.0, 3081.5, 3082.0, 3082.5, 3083.0, 3083....","[2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ..."


In [250]:
state.our_trades.tail(8)

,internal_log_id,strategy_time,direction,order_id,executed_qty,px
40,138,2024-04-12 19:39:05.243816064,Sell,45755241673,1,3081.5
41,140,2024-04-12 19:40:26.370759803,Buy,45755265579,3,3080.0
42,142,2024-04-12 19:40:27.168957116,Buy,45755265825,1,3080.5
43,145,2024-04-12 19:40:27.977960933,Buy,45755265609,1,3080.5
44,147,2024-04-12 19:40:28.458686743,Buy,45755265925,2,3081.0
45,148,2024-04-12 19:40:28.822034250,Buy,45755266008,1,3081.0
46,151,2024-04-12 19:40:29.785514470,Buy,45755265706,1,3080.5
47,152,2024-04-12 19:40:29.923505273,Buy,R249383740,3,3080.0


In [252]:
state.orders[state.orders.order_id == 'R249383740']

,internal_log_id,strategy_time,order_id,direction,px,qty
3397,141,2024-04-12 19:40:27.168716516,R249383740,Buy,3080.0,3
3423,142,2024-04-12 19:40:27.169328616,R249383740,Buy,3080.0,3
3450,143,2024-04-12 19:40:27.604465025,R249383740,Buy,3080.0,3
3478,144,2024-04-12 19:40:27.977748033,R249383740,Buy,3080.0,3
3505,145,2024-04-12 19:40:27.978329733,R249383740,Buy,3080.0,3
3533,146,2024-04-12 19:40:28.458524443,R249383740,Buy,3080.0,3
3561,147,2024-04-12 19:40:28.459234043,R249383740,Buy,3080.0,3


In [195]:
# def plot_target_bid_ask(state: StrategyState):
#     target_bid_ask = state.target_bid_ask

#     return target_bid_ask


# plot_target_bid_ask(state)